# Motion & Speed detection using openCV library
1. Import a video stream
2. The application track different models / objects in the videos. Draw one bounding box around the object and show the speed of the box

In [3]:
import numpy as np
import cv2

def video(image, video):
    # read image patch to detect the moving train  
    train_patch = cv2.imread(image, cv2.IMREAD_COLOR)

    #reshape the image(the number of color, width of patch, height of patch)
    nc, wPatch, hPatch = train_patch.shape[::-1]
    # split the patch only with red color 
    _, _, red_patch = cv2.split(train_patch)
    
    # capture the video 
    cap = cv2.VideoCapture(video)
    # we have ret and frame being defined as the cap.read()
    # ret is a boolean regarding whether or not there was a return at all, at the frame is each frame that is returned.
    ret, frame_past = cap.read()
    ret, frame_curr = cap.read()
    old_centroid = None
    
    while (cap.isOpened()):
        
        # extract red channel of current frame
        _, _, red_frame = cv2.split(frame_curr)

        # Perform matching operations on red channel
        res = cv2.matchTemplate(red_frame, red_patch, cv2.TM_CCOEFF_NORMED)
        max_similarity = np.max(res)
        # print(max_similarity)
        
        # similarity measure is above a threshold then train is detected
        if max_similarity > 0.50:
            loc = np.where(res >= max_similarity)
            patch_loc = (loc[1][0], loc[0][0])  # when drawing x and y are inverted
            cv2.rectangle(frame_curr, patch_loc, (patch_loc[0] + wPatch, patch_loc[1] + hPatch), (0, 255, 255), 10)
            
            new_centroid = np.array([patch_loc[0] + int(wPatch / 2), patch_loc[1] + int(hPatch / 2)])
            #cv2.circle(image, center_coordinates, radius, color, thickness)
            cv2.circle(frame_curr, tuple(new_centroid), 5, (0, 255, 255), -1)
            cv2.putText(frame_curr, "centroid", (new_centroid[0] - 25, new_centroid[1] - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (0, 255, 255), 2)
            
            if old_centroid is not None:
                # vector norm
                distance_run = np.linalg.norm(new_centroid - old_centroid, 2)  # in terms of pixels
                if distance_run > 5:
                    
                    # cv2.putText(image, text, org, font, fontScale, color, thickness)
                    cv2.putText(frame_curr, "Status: {}".format('moving'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 0, 255), 3)
                    
                    # 1pixel is 0.026cm, 30 frames per second 
                    speed = (distance_run*0.026)*30/100
                    cv2.putText(frame_curr, "Speed: {:.2f} m/s".format(speed), (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 0, 255), 3)
                
                else:
                    cv2.putText(frame_curr, "Status: {}".format('stopped'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0, 0, 255), 3)
            old_centroid = np.array(new_centroid)  # update old_centroid
            
            
        else:
            old_centroid = None  # update old_centroid
            cv2.putText(frame_curr, "Status: {}".format('No train'), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
            
        cv2.imshow('final result', frame_curr)
        ret, frame_curr = cap.read()
            
        if cv2.waitKey(1) & 0xFF == ord('q'): # to stop reading video by pressing 'q' on keyboard
            break
        
    cv2.destroyAllWindows()
    cap.release()

In [4]:
video('train.png', 'VID_20181016_143349.mp4' )

<img src='final result_screenshot.png'>